# Face Recognition Using CNN Architecture in Python

> Convolutional Neural Networks(CNN) has changed the way we used to learn images. CNN mimics the way humans see images, by focussing on one portion of the image at a time and scanning the whole image, this is called covolution operation.

> CNN boils down every image as a vector of numbers, which can be learned by the fully connected Dense layers of ANN.

**In this perticular case study I will be performing how to implement a face recognition model using CNN. You can use this template to create an image classification model on any group of images by putting them in a folder and creating a class.**

## About the Dataset (Images)

> The data contains cropped face images of 16 people divided into Training and testing. We will train the CNN model using the images in the Training folder and then test the model by using the unseen images from the testing folder, to check if the model is able to recognise the face number i.e label given to each face during training of the unseen images or not.

In [1]:
# import libraries 
import tensorflow
from tensorflow import keras

# Image Agumentation with Preprocessing

In [2]:
train_images = r"C:\Users\RAGUWING\Desktop\Resume\Additional Documents\Face Images\Final Training Images"

## Image Preprocessing using keras


# As we know deep-learning is hungry for data, the data we have is only limited. 
# so lets perform **Image Agumentation** to create different versions
# of the original image, which leads to a better model, since it learns
# on the good and bad mix of images.

from keras.preprocessing.image import ImageDataGenerator
train_gen = ImageDataGenerator(
    shear_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True
)
# No transformations are made on the test data
test_gen = ImageDataGenerator()

# Generating training data
training_data = train_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

# generating test data
testing_data = test_gen.flow_from_directory(
    train_images, 
    target_size = (100,100),
    batch_size = 30,
    class_mode = 'categorical'
)

# Printing class labels for each face
testing_data.class_indices

Found 244 images belonging to 16 classes.
Found 244 images belonging to 16 classes.


{'face1': 0,
 'face10': 1,
 'face11': 2,
 'face12': 3,
 'face13': 4,
 'face14': 5,
 'face15': 6,
 'face16': 7,
 'face2': 8,
 'face3': 9,
 'face4': 10,
 'face5': 11,
 'face6': 12,
 'face7': 13,
 'face8': 14,
 'face9': 15}

**If you observe, the above dictionary is having keys as face_names and values as numbers. We need to swap them because the classifier model will return the answer as the numeric mapping and we need to get the face_name out of it.**

> Also, since this is a multi-class classification problem, we are counting the number of unique faces, as that will be used as the number of output neurons in the output layer of fully connected ANN classifier.

## Mapping of class_labels with numeric value for classification

In [3]:
# training_data have numeric tag for each face
Train_class = training_data.class_indices

# lets store them in a dictionary with swap for future reference
Result_class = {}
for value_tag, face_tag in zip(Train_class.values(),Train_class.keys()):
    Result_class[value_tag] = face_tag

    
# use pickle to save the mapping's
import pickle
with open(r'C:\Users\RAGUWING\Desktop\Resume\Additional Documents\Face Images\ResultMap.pkl','wb') as Final_mapping:
    pickle.dump(Result_class,Final_mapping)

In [4]:
print("Mapping of Face and its numeric value",Result_class)

Mapping of Face and its numeric value {0: 'face1', 1: 'face10', 2: 'face11', 3: 'face12', 4: 'face13', 5: 'face14', 6: 'face15', 7: 'face16', 8: 'face2', 9: 'face3', 10: 'face4', 11: 'face5', 12: 'face6', 13: 'face7', 14: 'face8', 15: 'face9'}


In [5]:
Output_Neurons=len(Result_class)
print('\n The Number of output neurons: ', Output_Neurons)


 The Number of output neurons:  16


# Building the CNN Architecture, Model Compilation and Training

**In the below code snippet, I have created a CNN model with**

3. hidden layers of convolution
3. hidden layers of max pooling
1. layer of flattening
2. Hidden ANN layer
1. output layer with 16-neurons (one for each face)

In [6]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPool2D, Flatten, Dense

In [7]:
'''Initializing the Convolutional Neural Network'''
Model = Sequential()


''' STEP--1 Convolution
# Adding the first layer of CNN
# we are using the format (100,100,3) because we are using TensorFlow backend
# It means 3 matrix of size (100x100) pixels representing Red, Green and Blue components of pixels
'''

Model.add(Conv2D(16, kernel_size=(5,5), strides=(1,1), input_shape = (100,100,3),activation='relu'))

'''# Maxplooing layer'''

Model.add(MaxPool2D(pool_size=(2,2)))

'''Adding additional layers  of convolution and 
        maxpooling for better model accuracy and performance'''

Model.add(Conv2D(32,kernel_size=(3,3),strides=(1,1),activation='relu'))
Model.add(MaxPool2D(pool_size=(2,2)))

'''# Add a flatten layer to convert the vector to one dimensional'''
Model.add(Flatten())

'''Add dense layers and Initialize weights using 
                kernal initializer for better learing of image features and classification'''

Model.add(Dense(64,activation='relu'))
Model.add(Dense(Output_Neurons,activation='softmax'))

'''Perform Model Compilation'''

Model.compile(loss='categorical_crossentropy',optimizer = 'adam',metrics = ['Accuracy'])


'''# Using Early stopping to reduce the training time'''

from keras.callbacks import EarlyStopping
call = EarlyStopping(
                    min_delta=0.005,
                    patience=5,
                     verbose=1
                    )

import time
# Measuring the time taken by the model to train
StartTime=time.time()

'''# Model Training'''
Model.fit_generator(training_data,
          epochs = 30,
          validation_data=testing_data,
                   callbacks=call)

Endtime = time.time()
print('Total Training Time taken: ',round((Endtime-StartTime)/60),'Minutes')

C:\Users\RAGUWING\AppData\Local\Temp\ipykernel_11848\45880348.py:51: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  Model.fit_generator(training_data,


Epoch 1/30


9/9 [==============================] - 3s 198ms/step - loss: 114.9584 - Accuracy: 0.0615 - val_loss: 12.9830 - val_Accuracy: 0.1598
Epoch 2/30
9/9 [==============================] - 2s 177ms/step - loss: 5.1518 - Accuracy: 0.2500 - val_loss: 2.1260 - val_Accuracy: 0.3607
Epoch 3/30
9/9 [==============================] - 2s 199ms/step - loss: 2.0824 - Accuracy: 0.4262 - val_loss: 1.6130 - val_Accuracy: 0.5164
Epoch 4/30
9/9 [==============================] - 2s 174ms/step - loss: 1.5256 - Accuracy: 0.5410 - val_loss: 0.8518 - val_Accuracy: 0.7582
Epoch 5/30
9/9 [==============================] - 2s 176ms/step - loss: 0.7006 - Accuracy: 0.8033 - val_loss: 0.2181 - val_Accuracy: 0.9344
Epoch 6/30
9/9 [==============================] - 2s 173ms/step - loss: 0.3223 - Accuracy: 0.9057 - val_loss: 0.0873 - val_Accuracy: 0.9590
Epoch 7/30
9/9 [==============================] - 2s 172ms/step - loss: 0.2421 - Accuracy: 0.9180 - val_loss: 0.0437 - val_Accuracy: 0.9959
Epoch 8/30
9/9 

## Using the Classifier to make predictions on unseen test images

In [19]:
import numpy as np
from keras.preprocessing import image
import os
import glob

In [20]:
'''########### Making single predictions ###########'''

ImagePath=r"C:\Users\RAGUWING\Desktop\Resume\Additional Documents\Face Images\Final Training Images\face4\image_0054_Face_1.jpg"
test_image=image.load_img(ImagePath,target_size=(100, 100))
test_image=image.img_to_array(test_image)
 
test_image=np.expand_dims(test_image,axis=0)
 
result=Model.predict(test_image,verbose=0)
#print(training_set.class_indices)
 
print('####'*10)
print('Prediction is: ',Result_class[np.argmax(result)])


########################################
Prediction is:  face4


In [23]:
'''############ Making multiple predictions ###########'''

## Loading all the image paths from final testing folder for prediction
main_ = r"C:\Users\RAGUWING\Desktop\Resume\Additional Documents\Face Images\Final Testing Images"
img_paths = glob.glob(os.path.join(main_,'**','*.jpg'))

print(img_paths[0:5]) # every image will be a PIL object
print('*'*50)

for path in img_paths:
    test_image = image.load_img(path,target_size=(100,100))
    test_image = image.img_to_array(test_image)
    test_image = np.expand_dims(test_image,axis =0)
    result = Model.predict(test_image,verbose=0)
    print('Prediction: ',Result_class[np.argmax(result)])

['C:\\Users\\RAGUWING\\Desktop\\Resume\\Additional Documents\\Face Images\\Final Testing Images\\face1\\1face1.jpg', 'C:\\Users\\RAGUWING\\Desktop\\Resume\\Additional Documents\\Face Images\\Final Testing Images\\face1\\2face1.jpg', 'C:\\Users\\RAGUWING\\Desktop\\Resume\\Additional Documents\\Face Images\\Final Testing Images\\face1\\3face1.jpg', 'C:\\Users\\RAGUWING\\Desktop\\Resume\\Additional Documents\\Face Images\\Final Testing Images\\face1\\4face1.jpg', 'C:\\Users\\RAGUWING\\Desktop\\Resume\\Additional Documents\\Face Images\\Final Testing Images\\face10\\1face10.jpg']
**************************************************
Prediction:  face1
Prediction:  face1
Prediction:  face13
Prediction:  face13
Prediction:  face10
Prediction:  face10
Prediction:  face10
Prediction:  face10
Prediction:  face11
Prediction:  face11
Prediction:  face11
Prediction:  face11
Prediction:  face12
Prediction:  face12
Prediction:  face12
Prediction:  face12
Prediction:  face13
Prediction:  face13
Predicti